# IE 306.02 Assignment 1
In this assignment we have implemented a call center simulation. Clients place a call to front desk operator who then further forwards them to an expert operator. Clients may renage after front desk operator while waiting for expert, expert takes a break sometimes and both queues are FCFS. Exact details of both queues can be found in assignment description.

We have designed the system as two M/M/1 queues put one after another. In the first queue, server is front desk opetor who we modeled as a resource in the SimPy environment. Similarly, expert is the server in the second queue who is also modeled as a resource in SimPy. 

Below, we first describe our code and later interpret the statistics and results we have gathered from the simulation.

First we import the basic libraries.

In [1]:
import simpy
import random
import numpy.random
import numpy as np
from math import log,sqrt
import itertools # for easy list operations
from collections import namedtuple

A little helper function for formatting seconds to desired time format.

In [2]:
def format_time(total_minutes):
    secs = int((total_minutes % 1)*60)
    mins = int(total_minutes % 60)
    hours = int(total_minutes //60)
    return f" {hours:02}:{mins:02}:{secs:02}"
        
# an object to hold expert queue statistics
Expert_Q_Stat = namedtuple('Expert_Q_Stat', ['enter_e_q', 'exit_e_q'])

Below, we first describe our code and then interpret results.

We ollect and report statistics on: 

* Utilization of the front-desk operator

* Utilization of the expert operators, 

* Average Total Waiting Time,

* `Maximum Total Waiting Time` to `Total System Time Ratio`,

* Average number of people waiting to be served by the expert operator.

Global constants that will be used in the code.

In [3]:
# interarrival rates are exponentially distributed with mean 14.3 min
INTERARRIVAL_MEAN = 14.3 
# service time of first operator is LogNormal distributed with mean 7.2 and sd 2.7
m = 7.2
v = 2.7

FIRST_SERVICE_MEAN = log((m**2)/sqrt(v+m**2))
FIRST_SERVICE_STD = sqrt(log(v/(m**2)+1))


# customers leave queue if not serviced in exponentially distributed time with mean 60 min
RENEGE_TIME_MEAN = 60
# service time of expert is exponentially distributed with mean 10.2
EXPERT_SERVICE_MEAN = 10.2
# number of breaks taken by expert is Poisson distributed with mean 8 breaks
BREAKS_MEAN = 8
BREAK_DUR = 3

# number of customers simulation will be run for
CALLNUM1=1000
CALLNUM2=5000

LAST_EXIT = 0

LAST_CUSTOMER = 1

CUSTOMER_NUM=CALLNUM2


#RANDOM_SEED = 971
#random.seed(RANDOM_SEED)

Here, we define the necessary set of arrays for bookkeeping

In [4]:
# service times for front-desk operator
first_service_times = [] 
# service times for expert 
second_service_times = []
# time spent by a customer while it waits for the operator (Queue waiting time Wq)
first_queue_w_times = [] 
second_queue_w_times = [] 


expert_queue_log = []

# counter for breaks taken by the expert
break_count = 0



A said above, we modeled the front desk and expert operators as resources, calls(called customer in code) and breaks that expert takes are modeled as processes. Breaks taken by the expert makes the expert resource unavailable for 3 mins, so it made sense to model breaks as processes that try to get expert resource.

Class definition for breaks. Breaks are generated for expert operator. When they happen, they take 3 minutes long and expert is not available at that time. 

In [5]:
class Break:
    def __init__(self, env, expert):
        self.env = env
        self.expert = expert
        # it only action is happen, which can be thought as just being initiated
        self.action = env.process(self.happen())
    
    
    def happen(self):
        print(f"Break is on the way \t\t\t\t at {format_time(self.env.now)}")
        with self.expert.request() as req:
            yield req
            print(f"Break started \t\t\t\t\t at {format_time(self.env.now)}")
            yield self.env.timeout(BREAK_DUR)
            print(f"Break ended \t\t\t\t\t at {format_time(self.env.now)}")

* The class definition for the customers arriving at the modeled system. When they are created, they immediatelly initiate a call (i.e. activate the call process). 

* Once a call is initiated, this is registered as a request to the operator resource. The customer is put on hold until the resource activates it back. 

* When the resource is available, the customer is activated and it then initiates first the ask_question_operator process. The duration of a question-answer session is determined randomly according to a LogNormal distribution.

* After this, they are forwarded to expert operator. While waiting for the expert, they may renege after some time taken from an exponential distribution if they aren't addressed by the expert.

* Service time of the expert is taken from another exponential distribution.


In [6]:
class Customer(object):
    def __init__(self, _id, env, operator, expert):
        self.env = env
        self.id = _id
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
    
    
    def call(self):
        print(f"Cust {self.id:4} entered the \t\t FRONT Queue. \t at {format_time(self.arrival_t)}")
 
        with operator.request() as op_req :
            yield op_req
            self.wait_front = self.env.now - self.arrival_t
            print(f"Cust {self.id:4} started talking with \t FRONT Desk \t at {format_time(self.env.now)}")
            if self.wait_front:
                    print(f"Cust {self.id:4} waited in \t\t FRONT Queue \t for{format_time(self.wait_front)}")
            first_queue_w_times.append(self.wait_front)
            yield self.env.process(self.ask_question_operator())
            self.finish_operator=self.env.now
            print(f"Cust {self.id:4} entered the \t\t EXPERT Queue. \t at {format_time(self.finish_operator)}")
        
        if self.env.now >=60*8:
            #Shop is closed
            print(f"Cust {self.id:4} exited \t\t SUDDENLY \t at {format_time(self.env.now)}")
            print(f"-- shop is closed")
        else:
            with expert.request() as exp_req: 
                enter_e_q = self.env.now
                # customers may renege in the expert queue
                self.patience=random.expovariate(1/RENEGE_TIME_MEAN)
                results = yield exp_req | env.timeout(self.patience)
                self.wait_exp=self.env.now-self.finish_operator
                second_queue_w_times.append(self.wait_exp) 
                exit_e_q = self.env.now
                global expert_queue_log
                expert_queue_log.append(Expert_Q_Stat(enter_e_q, exit_e_q))

                if exp_req not in results:
                    #Leave queue, reneged
                    print(f"Cust {self.id:4} exited \t\t SUDDENLY \t at {format_time(self.env.now)}")
                    print(f"-- after waiting {format_time(self.wait_exp)}")

                else:
                # We got to the expert
                    print(f"Cust {self.id:4} started talking with \t EXPERT \t at {format_time(self.env.now)}")
                    if self.wait_exp:
                        print(f"Cust {self.id:4} waited in \t\t EXPERT Queue \t for{format_time(self.wait_exp)}")
                    yield self.env.process(self.ask_question_expert())
                    self.finish_expert=self.env.now
                    print(f"Cust {self.id:4} exited \t\t NORMALLY \t at {format_time(self.finish_expert)}")
        
        global LAST_EXIT
        LAST_EXIT = self.env.now
            
    def ask_question_operator(self):
        service_opr = random.lognormvariate(FIRST_SERVICE_MEAN,FIRST_SERVICE_STD)
        first_service_times.append(service_opr)
        yield self.env.timeout(service_opr)


    def ask_question_expert(self):
        service_exp = random.expovariate(1/EXPERT_SERVICE_MEAN)
        second_service_times.append(service_exp)
        yield self.env.timeout(service_exp)

Generator functions for customers and breaks. 

* Interrarrival times for both are taken from different exponential distributions. 

* Customer generator runs as many times as the number of total customers we want in our simulation, break generator runs through the whole shift.


In [10]:
def customer_generator(env, operator,expert,callnum):
    for i in range(callnum):
        global LAST_CUSTOMER
        LAST_CUSTOMER = LAST_CUSTOMER + 1
        #if env.now >= 60*8:
        #    break
        # lamda is how many for any minute(rate). 1/lamda is how many min for 1 event
        time_between_next_customer = random.expovariate(1/INTERARRIVAL_MEAN)
        customer = Customer(i+1, env, operator,expert)
        #if env.now + time_between_next_customer >= 60*8:
        #    break
        yield env.timeout(time_between_next_customer)
        
def break_generator(env, expert):
    while LAST_CUSTOMER<=CUSTOMER_NUM:
        time_to_break = random.expovariate(1/60)
        #if env.now + time_to_break >= 60*8:
        #    break
        yield env.timeout(time_to_break)
        Break(env, expert)
        global break_count
        break_count += 1

Here, we define our processes and resources and start running the simulation environment!

# To run the simulation for 1000 customers, make sure that CUSTOMER_NUM equals CALLNUM1

In [11]:

env = simpy.Environment()
operator = simpy.Resource(env, capacity = 1)
expert=simpy.Resource(env, capacity = 1)
env.process(customer_generator(env, operator,expert,CUSTOMER_NUM))
env.process(break_generator(env, expert))
end_simulation = simpy.Event(env)
env.run() 

Cust    1 entered the 		 FRONT Queue. 	 at  00:00:00
Cust    1 started talking with 	 FRONT Desk 	 at  00:00:00
Break is on the way 				 at  00:02:09
Break started 					 at  00:02:09
Cust    1 entered the 		 EXPERT Queue. 	 at  00:03:52
Break ended 					 at  00:05:09
Cust    1 started talking with 	 EXPERT 	 at  00:05:09
Cust    1 waited in 		 EXPERT Queue 	 for 00:01:17
Break is on the way 				 at  00:11:32
Cust    2 entered the 		 FRONT Queue. 	 at  00:17:04
Cust    2 started talking with 	 FRONT Desk 	 at  00:17:04
Cust    1 exited 		 NORMALLY 	 at  00:21:17
Break started 					 at  00:21:17
Cust    2 entered the 		 EXPERT Queue. 	 at  00:23:29
Break ended 					 at  00:24:17
Cust    2 started talking with 	 EXPERT 	 at  00:24:17
Cust    2 waited in 		 EXPERT Queue 	 for 00:00:48
Cust    2 exited 		 NORMALLY 	 at  00:27:27
Cust    3 entered the 		 FRONT Queue. 	 at  00:29:39
Cust    3 started talking with 	 FRONT Desk 	 at  00:29:39
Cust    3 entered the 		 EXPERT Queue. 	 at  00:35:2

Cust  389 exited 		 SUDDENLY 	 at  92:35:41
-- shop is closed
Cust  390 entered the 		 FRONT Queue. 	 at  92:46:45
Cust  390 started talking with 	 FRONT Desk 	 at  92:46:45
Cust  390 entered the 		 EXPERT Queue. 	 at  92:52:05
Cust  390 exited 		 SUDDENLY 	 at  92:52:05
-- shop is closed
Cust  391 entered the 		 FRONT Queue. 	 at  93:06:26
Cust  391 started talking with 	 FRONT Desk 	 at  93:06:26
Cust  392 entered the 		 FRONT Queue. 	 at  93:06:58
Cust  391 entered the 		 EXPERT Queue. 	 at  93:13:02
Cust  391 exited 		 SUDDENLY 	 at  93:13:02
-- shop is closed
Cust  392 started talking with 	 FRONT Desk 	 at  93:13:02
Cust  392 waited in 		 FRONT Queue 	 for 00:06:04
Cust  392 entered the 		 EXPERT Queue. 	 at  93:19:20
Cust  392 exited 		 SUDDENLY 	 at  93:19:20
-- shop is closed
Cust  393 entered the 		 FRONT Queue. 	 at  93:28:55
Cust  393 started talking with 	 FRONT Desk 	 at  93:28:55
Cust  394 entered the 		 FRONT Queue. 	 at  93:31:35
Cust  393 entered the 		 EXPERT Queue. 

Cust  829 entered the 		 EXPERT Queue. 	 at  197:25:46
Cust  829 exited 		 SUDDENLY 	 at  197:25:46
-- shop is closed
Cust  830 started talking with 	 FRONT Desk 	 at  197:25:46
Cust  830 waited in 		 FRONT Queue 	 for 00:06:20
Cust  831 entered the 		 FRONT Queue. 	 at  197:28:00
Cust  832 entered the 		 FRONT Queue. 	 at  197:28:15
Cust  830 entered the 		 EXPERT Queue. 	 at  197:30:34
Cust  830 exited 		 SUDDENLY 	 at  197:30:34
-- shop is closed
Cust  831 started talking with 	 FRONT Desk 	 at  197:30:34
Cust  831 waited in 		 FRONT Queue 	 for 00:02:34
Cust  833 entered the 		 FRONT Queue. 	 at  197:37:37
Break is on the way 				 at  197:38:17
Break started 					 at  197:38:17
Cust  831 entered the 		 EXPERT Queue. 	 at  197:38:38
Cust  831 exited 		 SUDDENLY 	 at  197:38:38
-- shop is closed
Cust  832 started talking with 	 FRONT Desk 	 at  197:38:38
Cust  832 waited in 		 FRONT Queue 	 for 00:10:22
Break ended 					 at  197:41:17
Cust  834 entered the 		 FRONT Queue. 	 at  197:4

Cust 1201 entered the 		 FRONT Queue. 	 at  284:09:43
Cust 1201 started talking with 	 FRONT Desk 	 at  284:09:43
Cust 1202 entered the 		 FRONT Queue. 	 at  284:12:06
Cust 1201 entered the 		 EXPERT Queue. 	 at  284:14:14
Cust 1201 exited 		 SUDDENLY 	 at  284:14:14
-- shop is closed
Cust 1202 started talking with 	 FRONT Desk 	 at  284:14:14
Cust 1202 waited in 		 FRONT Queue 	 for 00:02:07
Cust 1203 entered the 		 FRONT Queue. 	 at  284:15:53
Break is on the way 				 at  284:20:10
Break started 					 at  284:20:10
Break is on the way 				 at  284:20:23
Break ended 					 at  284:23:10
Break started 					 at  284:23:10
Cust 1202 entered the 		 EXPERT Queue. 	 at  284:23:16
Cust 1202 exited 		 SUDDENLY 	 at  284:23:16
-- shop is closed
Cust 1203 started talking with 	 FRONT Desk 	 at  284:23:16
Cust 1203 waited in 		 FRONT Queue 	 for 00:07:23
Break ended 					 at  284:26:10
Cust 1203 entered the 		 EXPERT Queue. 	 at  284:29:10
Cust 1203 exited 		 SUDDENLY 	 at  284:29:10
-- shop is c

Cust 1687 entered the 		 FRONT Queue. 	 at  397:59:38
Cust 1687 started talking with 	 FRONT Desk 	 at  397:59:38
Break is on the way 				 at  398:02:57
Break started 					 at  398:02:57
Cust 1688 entered the 		 FRONT Queue. 	 at  398:03:14
Break ended 					 at  398:05:57
Cust 1687 entered the 		 EXPERT Queue. 	 at  398:07:47
Cust 1687 exited 		 SUDDENLY 	 at  398:07:47
-- shop is closed
Cust 1688 started talking with 	 FRONT Desk 	 at  398:07:47
Cust 1688 waited in 		 FRONT Queue 	 for 00:04:33
Cust 1689 entered the 		 FRONT Queue. 	 at  398:11:29
Cust 1690 entered the 		 FRONT Queue. 	 at  398:11:34
Cust 1691 entered the 		 FRONT Queue. 	 at  398:12:36
Cust 1688 entered the 		 EXPERT Queue. 	 at  398:14:18
Cust 1688 exited 		 SUDDENLY 	 at  398:14:18
-- shop is closed
Cust 1689 started talking with 	 FRONT Desk 	 at  398:14:18
Cust 1689 waited in 		 FRONT Queue 	 for 00:02:49
Cust 1692 entered the 		 FRONT Queue. 	 at  398:16:05
Cust 1689 entered the 		 EXPERT Queue. 	 at  398:20:16


...

Cust 2967 exited 		 SUDDENLY 	 at  691:31:39
-- shop is closed
Cust 2968 entered the 		 FRONT Queue. 	 at  691:41:40
Cust 2968 started talking with 	 FRONT Desk 	 at  691:41:40
Cust 2969 entered the 		 FRONT Queue. 	 at  691:42:38
Cust 2970 entered the 		 FRONT Queue. 	 at  691:44:16
Cust 2968 entered the 		 EXPERT Queue. 	 at  691:51:14
Cust 2968 exited 		 SUDDENLY 	 at  691:51:14
-- shop is closed
Cust 2969 started talking with 	 FRONT Desk 	 at  691:51:14
Cust 2969 waited in 		 FRONT Queue 	 for 00:08:35
Cust 2969 entered the 		 EXPERT Queue. 	 at  691:58:24
Cust 2969 exited 		 SUDDENLY 	 at  691:58:24
-- shop is closed
Cust 2970 started talking with 	 FRONT Desk 	 at  691:58:24
Cust 2970 waited in 		 FRONT Queue 	 for 00:14:08
Cust 2971 entered the 		 FRONT Queue. 	 at  692:01:14
Break is on the way 				 at  692:04:35
Break started 					 at  692:04:35
Cust 2970 entered the 		 EXPERT Queue. 	 at  692:05:38
Cust 2970 exited 		 SUDDENLY 	 at  692:05:38
-- shop is closed
Cust 2971 star

Cust 3439 exited 		 SUDDENLY 	 at  816:19:43
-- shop is closed
Cust 3440 started talking with 	 FRONT Desk 	 at  816:19:43
Cust 3440 waited in 		 FRONT Queue 	 for 00:17:07
Cust 3440 entered the 		 EXPERT Queue. 	 at  816:26:11
Cust 3440 exited 		 SUDDENLY 	 at  816:26:11
-- shop is closed
Cust 3441 started talking with 	 FRONT Desk 	 at  816:26:11
Cust 3441 waited in 		 FRONT Queue 	 for 00:18:54
Cust 3441 entered the 		 EXPERT Queue. 	 at  816:32:14
Cust 3441 exited 		 SUDDENLY 	 at  816:32:14
-- shop is closed
Cust 3442 entered the 		 FRONT Queue. 	 at  816:54:33
Cust 3442 started talking with 	 FRONT Desk 	 at  816:54:33
Break is on the way 				 at  817:01:21
Break started 					 at  817:01:21
Cust 3442 entered the 		 EXPERT Queue. 	 at  817:02:52
Cust 3442 exited 		 SUDDENLY 	 at  817:02:52
-- shop is closed
Break is on the way 				 at  817:02:55
Cust 3443 entered the 		 FRONT Queue. 	 at  817:04:17
Cust 3443 started talking with 	 FRONT Desk 	 at  817:04:17
Break ended 					 at  8

Cust 3892 entered the 		 EXPERT Queue. 	 at  932:50:52
Cust 3892 exited 		 SUDDENLY 	 at  932:50:52
-- shop is closed
Break is on the way 				 at  932:53:53
Break started 					 at  932:53:53
Break ended 					 at  932:56:53
Cust 3893 entered the 		 FRONT Queue. 	 at  933:10:54
Cust 3893 started talking with 	 FRONT Desk 	 at  933:10:54
Cust 3893 entered the 		 EXPERT Queue. 	 at  933:15:05
Cust 3893 exited 		 SUDDENLY 	 at  933:15:05
-- shop is closed
Cust 3894 entered the 		 FRONT Queue. 	 at  933:25:30
Cust 3894 started talking with 	 FRONT Desk 	 at  933:25:30
Cust 3894 entered the 		 EXPERT Queue. 	 at  933:30:43
Cust 3894 exited 		 SUDDENLY 	 at  933:30:43
-- shop is closed
Cust 3895 entered the 		 FRONT Queue. 	 at  933:35:26
Cust 3895 started talking with 	 FRONT Desk 	 at  933:35:26
Cust 3895 entered the 		 EXPERT Queue. 	 at  933:41:50
Cust 3895 exited 		 SUDDENLY 	 at  933:41:50
-- shop is closed
Cust 3896 entered the 		 FRONT Queue. 	 at  933:49:41
Cust 3896 started talking w

Cust 4243 exited 		 SUDDENLY 	 at  1016:51:57
-- shop is closed
Cust 4244 started talking with 	 FRONT Desk 	 at  1016:51:57
Cust 4244 waited in 		 FRONT Queue 	 for 00:06:18
Cust 4246 entered the 		 FRONT Queue. 	 at  1016:56:01
Break is on the way 				 at  1016:57:44
Break started 					 at  1016:57:44
Cust 4244 entered the 		 EXPERT Queue. 	 at  1016:59:21
Cust 4244 exited 		 SUDDENLY 	 at  1016:59:21
-- shop is closed
Cust 4245 started talking with 	 FRONT Desk 	 at  1016:59:21
Cust 4245 waited in 		 FRONT Queue 	 for 00:08:24
Break ended 					 at  1017:00:44
Cust 4245 entered the 		 EXPERT Queue. 	 at  1017:05:40
Cust 4245 exited 		 SUDDENLY 	 at  1017:05:40
-- shop is closed
Cust 4246 started talking with 	 FRONT Desk 	 at  1017:05:40
Cust 4246 waited in 		 FRONT Queue 	 for 00:09:38
Cust 4246 entered the 		 EXPERT Queue. 	 at  1017:13:34
Cust 4246 exited 		 SUDDENLY 	 at  1017:13:34
-- shop is closed
Cust 4247 entered the 		 FRONT Queue. 	 at  1017:14:12
Cust 4247 started talking

Break ended 					 at  1105:46:07
Cust 4647 entered the 		 EXPERT Queue. 	 at  1105:52:16
Cust 4647 exited 		 SUDDENLY 	 at  1105:52:16
-- shop is closed
Break is on the way 				 at  1105:52:19
Break started 					 at  1105:52:19
Cust 4648 entered the 		 FRONT Queue. 	 at  1105:55:09
Cust 4648 started talking with 	 FRONT Desk 	 at  1105:55:09
Break ended 					 at  1105:55:19
Cust 4649 entered the 		 FRONT Queue. 	 at  1105:58:34
Cust 4648 entered the 		 EXPERT Queue. 	 at  1106:01:01
Cust 4648 exited 		 SUDDENLY 	 at  1106:01:01
-- shop is closed
Cust 4649 started talking with 	 FRONT Desk 	 at  1106:01:01
Cust 4649 waited in 		 FRONT Queue 	 for 00:02:26
Cust 4650 entered the 		 FRONT Queue. 	 at  1106:07:17
Cust 4649 entered the 		 EXPERT Queue. 	 at  1106:07:30
Cust 4649 exited 		 SUDDENLY 	 at  1106:07:30
-- shop is closed
Cust 4650 started talking with 	 FRONT Desk 	 at  1106:07:30
Cust 4650 waited in 		 FRONT Queue 	 for 00:00:13
Cust 4651 entered the 		 FRONT Queue. 	 at  1106:11

Lastly, we print our statistics we gathered from the simulation

In [12]:
util_front = sum(first_service_times) / LAST_EXIT
util_exp = sum(second_service_times) / LAST_EXIT

total_waits = [f_w + e_w for f_w, e_w in itertools.zip_longest(
    first_queue_w_times, 
    second_queue_w_times, 
    fillvalue=0
)]

total_services = [f_s + e_s for f_s, e_s in itertools.zip_longest(
    first_service_times,
    second_service_times,
    fillvalue=0
)]

total_system = [w + s for w, s in itertools.zip_longest(
    total_waits,
    total_services,
    fillvalue=0
)]

# this makes sure everything went smoothly 
assert(len(total_waits) == len(total_system))

max_ratio = max((w/sys if sys else 0 for w, sys in zip(total_waits, total_system)))

# a helper function to calculate avg length of expert queue
def log_to_avg_len(log):
   ins = sorted((entry.enter_e_q for entry in log))
   outs = sorted((entry.exit_e_q for entry in log))
   significant_points = np.array(sorted(ins + outs))
   in_mask = np.array([i in ins for i in significant_points])
   out_mask = np.array([o in outs for o in significant_points])
   queue_len = np.zeros(len(significant_points))

   
   for i, is_in in enumerate(in_mask):
      if is_in:
         queue_len[i:] += 1

   for i, is_in in enumerate(out_mask):
      if is_in:
         queue_len[i:] -= 1

   timedeltas = np.copy(significant_points)
   for i, elem in enumerate(timedeltas):
      timedeltas[i+1:] -= elem

   return np.sum( timedeltas*queue_len ) / outs[-1]

stats = {
    "break_count": break_count,
    "util_front": util_front,
    "util_exp": util_exp,
    "avg_total_wait": sum(total_waits) / len(total_waits),
    "max_wait_2_sys_ratio": max_ratio,
    "avg_len_of_exp_q": log_to_avg_len(expert_queue_log),
}

print("Number of breaks taken: %d" % (stats["break_count"]))
print("Utilization of front desk operator: %%%f" % (stats["util_front"]))
print("Utilization of expert operator: %%%f" % (stats["util_exp"]))
print("Average total waiting time: %f" % (stats["avg_total_wait"]))
print("Maximum of total waiting time to system time ratios: %%%f" % (stats["max_wait_2_sys_ratio"]))
print("Average length of expert waiting queue: %f" % (stats["break_count"]))


Utilization of front desk operator: %0.503797
Utilization of expert operator: %0.003811
Average total waiting time: 3.735215
Maximum of total waiting time to system time ratios: %0.879728
Average length of expert waiting queue: 1207.000000


In [ ]:
print (len(first_queue_w_times))

In [ ]:
print (len(second_queue_w_times))

In [ ]:
print(format_time(env.now))
print(format_time(LAST_EXIT))